In [5]:
divmod(-5,-3),bool(1),int(bool(-1)),bool(0),1&-1,True & False

((1, -2), True, 1, False, 1, False)

In [1]:
class Node:

    def __init__(self):
        self.val = False  # True:覆盖 False:没覆盖 None:未标记
        self.left = None
        self.right = None
        self.lazy = None


class RangeModule:

    def __init__(self):
        self.root = Node()

    def addRange(self, left: int, right: int) -> None:
        self.updateTree(self.root, 1, 10**9, left, right - 1, True)

    def queryRange(self, left: int, right: int) -> bool:
        return self.queryTree(self.root, 1, 10**9, left, right - 1)

    def removeRange(self, left: int, right: int) -> None:
        self.updateTree(self.root, 1, 10**9, left, right - 1, False)

    def pushUp(self, node):
        node.val = node.left.val and node.right.val

    def pushDown(self, node):
        if not node.left: node.left = Node()
        if not node.right: node.right = Node()
        if node.lazy is None:
            return
        node.left.val = node.val
        node.left.lazy = node.lazy
        node.right.val = node.val
        node.right.lazy = node.lazy
        node.lazy = None

    def updateTree(self, node, si, ei, l, r, val):
        if ei < l or si > r:
            return
        if l <= si and ei <= r:
            node.val = val
            node.lazy = val
            return
        mi = si + ei >> 1
        self.pushDown(node)
        if l <= mi:  # 左树[si,mi]
            self.updateTree(node.left, si, mi, l, r, val)
        if mi + 1 <= r:
            self.updateTree(node.right, mi + 1, ei, l, r, val)
        self.pushUp(node)

    def queryTree(self, node, si, ei, l, r):
        if ei < l or si > r:
            return False
        if l <= si and ei <= r:
            return node.val
        mi = si + ei >> 1
        self.pushDown(node)
        ans = True
        if l <= mi:
            ans &= self.queryTree(node.left, si, mi, l, r)
        if mi + 1 <= r:
            ans &= self.queryTree(node.right, mi + 1, ei, l, r)
        return ans


# Your RangeModule object will be instantiated and called as such:
# obj = RangeModule()
# obj.addRange(left,right)
# param_2 = obj.queryRange(left,right)
# obj.removeRange(left,right)

In [2]:
class Node:

    def __init__(self, si, ei) -> None:
        self.si = si
        self.ei = ei
        self.mi = si + ei >> 1
        self.lgh = ei - si + 1
        self.val = 0  # 区间和
        self.lazy = None  # 区间每个值设为lazy 但未下传
        self.left = None
        self.right = None


class SegmentTree:

    def __init__(self, si, ei) -> None:
        self.root = Node(si, ei)

    def pushDown(self, node: Node):
        if not node.left: node.left = Node(node.si, node.mi)
        if not node.right: node.right = Node(node.mi + 1, node.ei)
        if node.lazy is None:
            return
        node.left.val = node.left.lgh * node.lazy
        node.right.val = node.right.lgh * node.lazy
        node.left.lazy = node.lazy
        node.right.lazy = node.lazy
        node.lazy = None

    def update(self, node: Node, l, r, val):  # [l,r]全部值设为val
        if node.ei < l or node.si > r:
            return
        if l <= node.si <= node.ei <= r:
            node.val = node.lgh * val
            node.lazy = val
            return
        self.pushDown(node)
        if node.left.ei >= l:
            self.update(node.left, l, r, val)
        if node.right.si <= r:
            self.update(node.right, l, r, val)
        node.val = node.left.val + node.right.val

    def query(self, node: Node, l, r):  # 查询[l,r]的和
        if node.ei < l or node.si > r:
            return 0
        if l <= node.si <= node.ei <= r:
            return node.val
        self.pushDown(node)
        ans = 0
        if node.left.ei >= l:
            ans += self.query(node.left, l, r)
        if node.right.si <= r:
            ans += self.query(node.right, l, r)
        return ans


class RangeModule:

    def __init__(self):
        self.st = SegmentTree(1, 10**9 - 1)

    def addRange(self, left: int, right: int) -> None:
        self.st.update(self.st.root, left, right - 1, 1)

    def queryRange(self, left: int, right: int) -> bool:
        return self.st.query(self.st.root, left, right - 1) == right - left

    def removeRange(self, left: int, right: int) -> None:
        self.st.update(self.st.root, left, right - 1, 0)


# Your RangeModule object will be instantiated and called as such:
# obj = RangeModule()
# obj.addRange(left,right)
# param_2 = obj.queryRange(left,right)
# obj.removeRange(left,right)

In [3]:
class Node:

    def __init__(self, si, ei) -> None:
        self.si = si
        self.ei = ei
        self.mi = si + ei >> 1
        self.val = False  # 区间覆盖情况值
        self.lazy = None  # 区间每个值设为lazy 但未下传
        self.left = None
        self.right = None


class SegmentTree:

    def __init__(self, si, ei) -> None:
        self.root = Node(si, ei)

    def pushDown(self, node: Node):
        if not node.left: node.left = Node(node.si, node.mi)
        if not node.right: node.right = Node(node.mi + 1, node.ei)
        if node.lazy is None:
            return
        node.left.val = node.lazy
        node.right.val = node.lazy
        node.left.lazy = node.lazy
        node.right.lazy = node.lazy
        node.lazy = None

    def update(self, node: Node, l, r, val):  # [l,r]全部值设为val
        if node.ei < l or node.si > r:
            return
        if l <= node.si <= node.ei <= r:
            node.val = val
            node.lazy = val
            return
        self.pushDown(node)
        if node.left.ei >= l:
            self.update(node.left, l, r, val)
        if node.right.si <= r:
            self.update(node.right, l, r, val)
        node.val = node.left.val & node.right.val

    def query(self, node: Node, l, r):  # 查询[l,r]的和
        if node.ei < l or node.si > r:
            return 0
        if l <= node.si <= node.ei <= r:
            return node.val
        self.pushDown(node)
        ans = True
        if node.left.ei >= l:
            ans &= self.query(node.left, l, r)
        if node.right.si <= r:
            ans &= self.query(node.right, l, r)
        return ans


class RangeModule:

    def __init__(self):
        self.st = SegmentTree(1, 10**9 - 1)

    def addRange(self, left: int, right: int) -> None:
        self.st.update(self.st.root, left, right - 1, True)

    def queryRange(self, left: int, right: int) -> bool:
        return self.st.query(self.st.root, left, right - 1)

    def removeRange(self, left: int, right: int) -> None:
        self.st.update(self.st.root, left, right - 1, False)


# Your RangeModule object will be instantiated and called as such:
# obj = RangeModule()
# obj.addRange(left,right)
# param_2 = obj.queryRange(left,right)
# obj.removeRange(left,right)

In [4]:
class Node:

    def __init__(self, track=False, lazy=None, lc=None, rc=None):
        self.track = track  # 表示当前区间所有元素是否被追踪
        self.lazy = lazy  # 表示当前区间的所有子区间应该更新的追踪状态，None表示子区间无需更新
        self.lc = lc
        self.rc = rc


class RangeModule:

    def __init__(self):
        self.tree = Node()

    def pushup(self, node):
        node.track = bool(node.lc and node.lc.track and node.rc and node.rc.track)

    def pushdown(self, node):
        if not node.lc:
            node.lc = Node()
        if not node.rc:
            node.rc = Node()
        if node.lazy is not None:
            node.lc.track = node.lc.lazy = node.rc.track = node.rc.lazy = node.lazy
            node.lazy = None

    def _addRange(self, tl, tr, node, l, r):  # tl,tr表示目标区间，l,r表示当前结点对应区间
        if tl > r or tr < l:
            return
        if tl <= l <= r <= tr:
            node.track = node.lazy = True
            return
        mid = (l + r) >> 1
        self.pushdown(node)
        self._addRange(tl, tr, node.lc, l, mid)
        self._addRange(tl, tr, node.rc, mid + 1, r)
        self.pushup(node)

    def addRange(self, left: int, right: int) -> None:
        self._addRange(left, right - 1, self.tree, 0, 10**9)

    def _queryRange(self, tl, tr, node, l, r):
        if tl > r or tr < l:
            return True
        if tl <= l <= r <= tr:
            return node.track
        mid = (l + r) >> 1
        self.pushdown(node)
        return self._queryRange(tl, tr, node.lc, l, mid) and self._queryRange(tl, tr, node.rc, mid + 1, r)

    def queryRange(self, left: int, right: int) -> bool:
        return self._queryRange(left, right - 1, self.tree, 0, 10**9)

    def _removeRange(self, tl, tr, node, l, r):
        if tl > r or tr < l:
            return
        if tl <= l <= r <= tr:
            node.track = node.lazy = False
            return
        self.pushdown(node)
        mid = (l + r) >> 1
        self._removeRange(tl, tr, node.lc, l, mid)
        self._removeRange(tl, tr, node.rc, mid + 1, r)
        self.pushup(node)

    def removeRange(self, left: int, right: int) -> None:
        self._removeRange(left, right - 1, self.tree, 0, 10**9)


In [ ]:
class Node:

    def __init__(self, si, ei) -> None:
        self.si = si
        self.ei = ei
        self.mi = si + ei >> 1
        self.val = False  # 区间覆盖情况值
        self.lazy = None  # 区间每个值设为lazy 但未下传
        self.left = None
        self.right = None


class SegmentTree:

    def __init__(self, si, ei) -> None:
        self.root = Node(si, ei)

    def pushDown(self, node: Node):
        if not node.left: node.left = Node(node.si, node.mi)
        if not node.right: node.right = Node(node.mi + 1, node.ei)
        if node.lazy is None:
            return
        node.left.val = node.lazy
        node.right.val = node.lazy
        node.left.lazy = node.lazy
        node.right.lazy = node.lazy
        node.lazy = None

    def update(self, node: Node, l, r, val):  # [l,r]全部值设为val
        if node.ei < l or node.si > r:
            return
        if l <= node.si <= node.ei <= r:
            node.val = val
            node.lazy = val
            return
        self.pushDown(node)
        self.update(node.left, l, node.left.ei, val)
        self.update(node.right, node.right.si, r, val)
        node.val = node.left.val & node.right.val

    def query(self, node: Node, l, r):  # 查询[l,r]的和
        if node.ei < l or node.si > r:
            return 0
        if l <= node.si <= node.ei <= r:
            return node.val
        self.pushDown(node)
        ans = True
        ans &= self.query(node.left, l, node.left.ei)
        ans &= self.query(node.right, node.right.si, r)
        return ans


class RangeModule:

    def __init__(self):
        self.st = SegmentTree(1, 10**9 - 1)

    def addRange(self, left: int, right: int) -> None:
        self.st.update(self.st.root, left, right - 1, True)

    def queryRange(self, left: int, right: int) -> bool:
        return self.st.query(self.st.root, left, right - 1)

    def removeRange(self, left: int, right: int) -> None:
        self.st.update(self.st.root, left, right - 1, False)


# Your RangeModule object will be instantiated and called as such:
# obj = RangeModule()
# obj.addRange(left,right)
# param_2 = obj.queryRange(left,right)
# obj.removeRange(left,right)